In [2]:
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np

In the below step we will provide the dataset and the labels to categories as 0 and 1

In [3]:
path = 'dataset'
categories = os.listdir(path)  #2 categories in the data ie. no mask and with mask

labels = [i for i in range(len(categories))]    #so if there are 2 categories so len(categories) is 2 and "i" will be 0 & 1

label_dict = dict(zip(categories,labels))

print(label_dict)
print(categories)
print(labels)

{'No_Mask': 0, 'With_Mask': 1}
['No_Mask', 'With_Mask']
[0, 1]


In [4]:
img_size = 100
Image_data = []
Image_category = []   #that is with mask or without mask

for category in categories:    # categories variable is define in the above cell
    
    folder_path = os.path.join(path,category)
    image_names = os.listdir(folder_path)
    
    for img_name in image_names:
        
        image_path = os.path.join(folder_path,img_name)
        img = cv2.imread(image_path)
        
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        #CONVERTUNG THE IMAGE INTO GREY SCALE
        
        resized = cv2.resize(img,(img_size, img_size))
        #resizing the grayscale into 100X100 as img_size = 100
        
        Image_data.append(resized)
        Image_category.append(label_dict[category])
        #appending the image into Image_data and the label into the Image_category 

In [5]:
np.save('Image_data',Image_data)
np.save('Category',new_category)

## TRAINING THE MODEL

In [3]:
import numpy as np
data = np.load('Image_data.npy')
target = np.load('Category.npy')
#loading the numpy array save in the previous(1.0 PROCESSING) notebook

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation

Using TensorFlow backend.


In [ ]:
#initializing cnn
model = Sequential()
data.shape

In [ ]:
#step 1 Convolution

#Conv2D(no. of feature maps you want , filter_size you want, image input size you want)
model.add(Conv2D(200, (3,3), input_shape = data.shape[1:]))
model.add(Activation('relu'))

#step2 MaxPooling
model.add(MaxPooling2D(pool_size = (2,2)))

In [ ]:
#adding one more convolutional layer and maxpooling layer to increase the accuracy

model.add(Conv2D(100, (3,3), activation = 'relu' ))
model.add(MaxPooling2D(pool_size = (2,2)))

In [ ]:
#step 3 flattening 

model.add(Flatten()) #it will open the matrix and create the inputs for the ann
model.add(Dropout(0.5))

In [ ]:
# Step 4 Full Connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(2, activation = 'softmax'))

In [ ]:
#compiling
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
data.shape

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=.20)

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('model_fmd', monitor = 'val_loss',verbose = 0, save_best_only = True, mode = 'auto')
hist = model.fit(train_data,train_target, epochs = 20, callbacks = [checkpoint], validation_split = 0.2)

In [ ]:
import matplotlib.pyplot as plt

plt.plot( hist.history["loss"], label="train_loss")
plt.plot(hist.history["val_loss"], label="val_loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

In [ ]:
print(model.evaluate(test_data,test_target))